# LLM - Evaluation: Unit Test


Install packages


In [1]:
!uv pip install -q \
    litellm==1.78.5 \
    python-dotenv==1.1.1 \
    pydantic==2.12.3

Import packages


In [ ]:
import io
import json
from unittest import TestCase, TestLoader, TestSuite, TextTestRunner

import litellm  # type: ignore
from dotenv import load_dotenv  # type: ignore
from pydantic import BaseModel  # type: ignore

load_dotenv()

True

Define structured output


In [ ]:
class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

Define llm completion


In [ ]:
def get_calendar_event(message):
    messages = [
        {"role": "system", "content": "Extract the event information."},
        {"role": "user", "content": message},
    ]

    completion = litellm.completion(
        model="gemini/gemini-2.0-flash",
        messages=messages,
        response_format=CalendarEvent,
    )

    content = completion.choices[0].message.content

    data = json.loads(content)

    return CalendarEvent.model_validate(data)

Create a test case


In [ ]:
class TestGetCalendarEvent(TestCase):
    def test_calendar_event(self):
        # Arrange
        message = "Alice and Bob are going to a science fair on friday."

        # Act
        calendar_event = get_calendar_event(message)

        # Assert
        self.assertIsInstance(calendar_event, CalendarEvent)
        self.assertIsInstance(calendar_event.participants, list)

        self.assertEqual(calendar_event.name.lower(), "science fair")
        self.assertEqual(calendar_event.date.lower(), "friday")
        self.assertEqual(calendar_event.participants, ["Alice", "Bob"])

- Create a test suit that will allow to run one or more test cases
- Run the test cases


In [ ]:
loader = TestLoader()
suite = TestSuite()

buffer = io.StringIO()
suite.addTest(loader.loadTestsFromTestCase(TestGetCalendarEvent))

TextTestRunner(stream=buffer, verbosity=2).run(suite)

output = buffer.getvalue()

print(output)

test_calendar_event (__main__.TestGetCalendarEvent.test_calendar_event) ... ok

----------------------------------------------------------------------
Ran 1 test in 1.775s

OK

